In [ ]:
import pandas as pd
import nltk
from collections import defaultdict
import random
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.core.display import display, Image


In [ ]:
df = pd.read_csv("wiki_data.csv", index_col='title')
df.head()

In [ ]:
df.loc["Тензорный анализ"]["text"]

In [ ]:
df["target"] = df.category.apply(lambda x: int(x ==  "Математика"))
del(df["category"])


In [ ]:
df.head(20)

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"[a-zа-яё]+")

def text2tokens(text, num=10):
    tokens = tokenizer.tokenize(text.lower())
    if len(tokens) > num:
        return tokens[:num]
    else:
        return ['#'] * (num - len(tokens)) +  tokens

In [ ]:
text2tokens('мама мыла раму')

In [ ]:
text2tokens(df.loc["Тензорный анализ"]["text"])

In [ ]:
token_ids = defaultdict(lambda: len(token_ids))

def text2tokenIds(text, num=10):
    tokens = text2tokens(text, num)
    result = []
    for token in tokens:
        result.append(token_ids[token])
    return np.array(result)

In [ ]:
text2tokenIds('мама мыла раму')

In [ ]:
text2tokenIds(df.loc["Тензорный анализ"]["text"], 10)

In [ ]:
max_len = 200
data = list(df.text.apply(lambda text: text2tokenIds(text, max_len)))  
X = pd.DataFrame(data, index = df.index)
y = df.target

max_features = len(token_ids)

In [ ]:
X

In [ ]:
display(Image("lstm.png"))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=96622)

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

evaluate_samples = 1000
attempt = 0
lstm_hidden_vars = 16
embedding_size = 32


model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=max_len))
model.add(LSTM(lstm_hidden_vars, return_sequences=True))
model.add(LSTM(lstm_hidden_vars))

model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))


model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])



In [ ]:
batch_size  = 256
epochs = 10

model.fit(
    X_train.values, y_train.values, 
    batch_size=batch_size, 
    validation_data=(X_test.values, y_test.values),
    epochs=epochs
)


In [111]:
def get_text_topics(text):
    tokens = text2tokenIds(text, max_len).reshape(1, max_len)
    math_prob = model.predict(tokens)
    chem_prob = 1 - math_prob
    
    return {"Математика": math_prob, "Химия": chem_prob}

In [112]:
text = """
Матема́тика (др.-греч. μᾰθημᾰτικά[1] < μάθημα «изучение; наука») 
— наука о структурах, порядке и отношениях, исторически сложившаяся на основе операций подсчёта, 
измерения и описания формы объектов[2]. 
Математические объекты создаются путём идеализации свойств реальных или других математических 
объектов и записи этих свойств на формальном языке. Математика не относится к естественным наукам, 
но широко используется в них как для точной формулировки их содержания, так и для получения новых результатов.
Математика — фундаментальная наука, предоставляющая (общие) языковые средства другим наукам;
тем самым она выявляет их структурную взаимосвязь и способствует нахождению самых общих законов природы[3].
"""
get_text_topics(text.lower())

{'Математика': array([[0.99309856]], dtype=float32),
 'Химия': array([[0.00690144]], dtype=float32)}

In [115]:
text="""
я люблю химию"""

In [116]:
get_text_topics(text.lower())

{'Математика': array([[0.991339]], dtype=float32),
 'Химия': array([[0.00866097]], dtype=float32)}

##### Ссылки для продлождения изучения

1. Генерация текстов https://www.tensorflow.org/tutorials/sequences/text_generation
1. Генерация изображений https://machinelearningmastery.com/how-to-caption-photos-with-deep-learning/
1. Машинный перевод https://medium.com/@umerfarooq_26378/neural-machine-translation-with-code-68c425044bbd
1. Распознавание речи https://zhuanlan.zhihu.com/p/28274740
1. Рекомендательная система https://towardsdatascience.com/introduction-to-recommender-system-part-2-adoption-of-neural-network-831972c4cbf7